In [1]:
import pandas as pd
from datetime import datetime
import astropack2 as ap2
from astropack2.tuning._tuner import hyperparameter_search
import os
from joblib import dump
from tqdm.notebook import tqdm

/home/usuario/anaconda3/envs/astrotest/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [3]:
# Caminho do arquivo de entrada
input_path = '../data/input_jpasgL.csv'
df_base = pd.read_csv(input_path)
df_base.set_index('ID', drop=True, inplace=True)

# Nome da coluna de índice (ou None para não usar)
index_col = 'ID'

# Survey e filtros
survey_filter = 'JPAS'
filters = ap2.FILTERS[survey_filter]

# Restrição de erro de magnitude
model_rest = '_01'  # '_01' = mais restrito, '_02' = menos restrito

# Tipo de modelo: 'RF' para Random Forest, 'XGB' para XGBoost
model_type = 'RF'

# Ajuste automático do sufixo do survey_train conforme o nome do arquivo de entrada
if 'A' in os.path.basename(input_path):
    survey_train = f'{survey_filter}A'
elif 'L' in os.path.basename(input_path):
    survey_train = f'{survey_filter}L'
else:
    survey_train = survey_filter
datetime_str = pd.Timestamp.now().strftime('%Y%m%d%H%M%S')

# Lista de parâmetros a serem processados
param_list = ['teff', 'logg', 'feh']

In [4]:
# Parâmetros para tuning (grade reduzida para teste rápido)
num_features = 20  # Número de features após pré-processamento
k_values = [k for k in [5, num_features] if k <= num_features]

param_dist = {
    'randomforestregressor__n_estimators': [10, 50],
    'randomforestregressor__max_features': [5, num_features],
    'randomforestregressor__min_samples_leaf': [1, 5],
    'randomforestregressor__bootstrap': [True],
    'randomforestregressor__max_depth': [None, 10],
    'randomforestregressor__min_samples_split': [2, 10],
    'selectkbest__k': k_values
}

# Parâmetros do RandomizedSearchCV para rodar rápido
n_iter = 2
cv = 2
n_jobs = -1
random_state = 42
test_size = 0.25
save_dir = 'pipeline'

In [5]:
# Função utilitária para buscar o nome correto da coluna no DataFrame
def get_column(df, aliases):
    for col in aliases:
        if col in df.columns:
            return col
    raise KeyError(f'Nenhuma coluna encontrada para: {aliases}')

# Aplicar filtros comuns uma vez só
df_common = df_base.copy()
max_err = 0.1 if model_rest == '_01' else 0.2  # filtro de erro de magnitude
error_columns = [f + '_err' for f in filters]

# Filtros de erro de magnitude (aplicados uma vez)
for col in error_columns:
    if col in df_common.columns:
        df_common = df_common[df_common[col] <= max_err]

# Filtros gerais (aplicados uma vez)
df_common = df_common.dropna(subset=['Dist'])


# Agora aplicar filtros específicos para cada parâmetro
preprocessed = {}
for param in param_list:
    aliases = ap2.PARAM_ALIASES[param]
    df = df_common.copy()
    
    # Filtros específicos por parâmetro
    if param.lower() == 'teff':
        param_col = get_column(df, aliases)
        df = df[df[param_col] <= 8300]
        err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_teff') or c.lower().endswith('teff_err'))]
        if err_col_candidates:
            err_col = err_col_candidates[0]
            df = df[df[err_col] < 300]
        df = df[df[param_col] != -9999]
        df = df.dropna(subset=[param_col])

    elif param.lower() in ['logg']:
        param_col = get_column(df, aliases)
        err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_'+param.lower()) 
                                                        or c.lower().endswith(param.lower()+'_err'))]
        if err_col_candidates:
            err_col = err_col_candidates[0]
            df = df[df[err_col] < 0.4]
        df = df[df[param_col] != -9999]
        df = df.dropna(subset=[param_col])
    
    elif param.lower() in ['feh']:
        param_col = get_column(df, aliases)
        err_col_candidates = [c for c in df.columns if (c.lower().startswith('e_'+param.lower()) 
                                                        or c.lower().endswith(param.lower()+'_err'))]
        if err_col_candidates:
            err_col = err_col_candidates[0]
            df = df[df[err_col] < 0.4]
        df = df[df[param_col] != -9999]
        df = df.dropna(subset=[param_col])
    

# Cálculo da magnitude absoluta específico para este parâmetro
    df = ap2.preprocess.calculate_abs_mag(df, filters, 'Dist')
    
    # Criação do dataframe de trabalho específico para este parâmetro
    work_df = ap2.preprocess.assemble_work_df(df, 
                                              filters=filters, 
                                              correction_pairs=None, 
                                              add_colors=True, 
                                              add_combinations=False)
    preprocessed[param] = {'df': df, 'work_df': work_df, 'param_col': param_col}



Iniciando processo de criação do dataframe de trabalho:

  - Adicionando cores ao dataframe... Tempo: 17.68 s

Processo finalizado! Shape da Tabela Gerada: (2060, 1653)
Iniciando processo de criação do dataframe de trabalho:

  - Adicionando cores ao dataframe... Tempo: 9.08 s

Processo finalizado! Shape da Tabela Gerada: (2060, 1653)
Iniciando processo de criação do dataframe de trabalho:

  - Adicionando cores ao dataframe... Tempo: 9.20 s

Processo finalizado! Shape da Tabela Gerada: (2073, 1653)


In [6]:
# Aplicar tuning para cada parâmetro usando os dados pré-processados
tuning_ids = []
os.makedirs('pipeline', exist_ok=True)
for param, data in tqdm(preprocessed.items(), desc='Tuning parâmetros'):
    X = data['work_df']
    Y = data['df'][data['param_col']]
    tuning_id = f'{datetime_str}_{survey_train}_{param}{model_rest}_{model_type}'
    best_pipeline, results = hyperparameter_search(
        X, Y,
        model_type=model_type,
        param_dist=param_dist,
        tuning_id=tuning_id,
        k_values=k_values,
        n_iter=n_iter,
        cv=cv,
        n_jobs=n_jobs,
        random_state=random_state,
        save_dir=save_dir
    )
    dump(best_pipeline, f'pipeline/{tuning_id}_pipeline.joblib')
    tuning_ids.append(tuning_id)
# Mostrar apenas os tuning_ids ao final
print('TUNING_IDs GERADOS:')
for tid in tuning_ids:
    print(tid)

Tuning parâmetros:   0%|          | 0/3 [00:00<?, ?it/s]

TUNING_IDs GERADOS:
20250923173244_JPASL_teff_01_RF
20250923173244_JPASL_logg_01_RF
20250923173244_JPASL_feh_01_RF
